In [1]:
import warnings 
warnings.filterwarnings('ignore')
from sklearn.datasets import load_iris
import numpy as np
import math
from tqdm import tqdm
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pandas as pd
from sklearn.preprocessing import (
    LabelEncoder,
    OneHotEncoder,
    OrdinalEncoder,
)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, ConfusionMatrixDisplay, confusion_matrix
import xgboost as xgb
import category_encoders.utils as util
from category_encoders import TargetEncoder
from sklearn.feature_extraction.text import CountVectorizer

Lectura de archivos

In [2]:
iden = pd.read_csv('train_identity.csv')

In [3]:
trans = pd.read_csv('train_transaction.csv')

Merge de df

In [4]:
joined = trans.merge(iden,how='left')

In [5]:
del(joined['TransactionID'])

Libero RAM

In [6]:
del iden

In [7]:
del trans

Busco el valor limite entre train y validation para hacer el split

In [8]:
limite = np.percentile(joined['TransactionDT'], 80)

Genero el DF de Validation

In [9]:
filtro_validation = joined['TransactionDT'] > limite

In [10]:
validation = joined[filtro_validation]

In [11]:
label_validation = validation[['isFraud']]

In [12]:
del(validation['isFraud'])

Genero el DF de train

In [13]:
filtro_train = joined['TransactionDT']<= limite

In [14]:
train = joined[filtro_train]

In [15]:
label_train = train[['isFraud']]

In [16]:
del(train['isFraud'])

Libero RAM


In [17]:
del joined

### Encodings

Creo un diccionario para almancenar los encodings.

In [18]:
dict_encoding = {}

Creo funciones para aplicar los encodings.

In [19]:
def countVect(df,columna,dic):
    if columna not in dic:
        dic[columna] = CountVectorizer(max_features =10)
        col_encoded = dic[columna].fit_transform(df[columna].fillna('desconocido')).astype(int).toarray()
    else:
        col_encoded = dic[columna].transform(df[columna].fillna('desconocido')).astype(int).toarray()
    col_encoded = pd.DataFrame(col_encoded).add_prefix(columna+ '_enconded_').fillna(0)
    del(df[columna])
    df_nuevo = pd.concat([df.reset_index(drop=True),col_encoded], axis=1)
    return df_nuevo

In [20]:
def one_hot_encode(df, columna, dict_encoding):
    if columna not in dict_encoding:
        dict_encoding[columna] = OneHotEncoder(handle_unknown='ignore')
        dict_encoding[columna].fit_transform(df[[columna]].astype(str)).astype(int)
    col_encoded = (dict_encoding[columna].transform(df[[columna]].astype(str)).astype(int))
    col_encoded = pd.DataFrame(col_encoded.todense()).add_prefix(columna + '_').fillna(0)
    del(df[columna])
    df_nuevo = pd.concat([df.reset_index(drop=True), col_encoded], axis=1)
    return df_nuevo

In [21]:
def mean_encoding_train(df,columna,label, dict_encoding):
    if columna not in dict_encoding:
        dict_encoding[columna] = TargetEncoder(handle_unknown='ignore')
    df[columna] = dict_encoding[columna].fit_transform(df[[columna]], label)
    df[columna].fillna(0,inplace=True)
    return df

In [22]:
def mean_encoding_val_test(df,columna, dict_encoding):
    df[columna] = dict_encoding[columna].transform(df[[columna]])
    df[columna].fillna(0,inplace=True)
    return df

In [23]:
def label_ec(df):
    for column in df.columns:
        if df[column].dtypes==object:
            dict_encoding[column] = OrdinalEncoder(handle_unknown='ignore')
            df[column] = dict_encoding[column].fit_transform(df[column].astype(str).values.reshape(-1, 1))
    if df[column].dtype==object:
        df[column] = dict_encoding[column].transform(df[column].astype(str).values.reshape(-1, 1))
    df.fillna(-1,inplace=True)
    return(df)

Defino los features que les aplicare los encodings: oneHot, mean o CountVectorizer

In [24]:
countV = ['id_30','id_31','DeviceInfo']

In [25]:
mean = ['addr2','DeviceType']

In [26]:
oneHot = ['P_emaildomain','card4','ProductCD','id_38','id_35','id_34']

### Modelo

In [27]:
xboost = xgb.XGBClassifier(random_state=0)

In [28]:
#xboost.get_xgb_params()

### Train

Realizo los encodings.

In [29]:
for i in countV:
    train=countVect(train,i,dict_encoding)

In [30]:
for i in oneHot:
    train = one_hot_encode(train,i,dict_encoding)

In [31]:
for i in mean:
    train = mean_encoding_train(train,i,label_train,dict_encoding)

In [32]:
train = label_ec(train)

Entreno el modelo

In [33]:
xboost.fit(train,label_train)

[15:49:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

### Validation


Realizo los encodings.

In [36]:
for i in countV:
    validation=countVect(validation,i,dict_encoding)

In [37]:
for i in oneHot:
    validation = one_hot_encode(validation,i,dict_encoding)

In [38]:
for i in mean:
    validation = mean_encoding_val_test(validation,i,dict_encoding)

In [39]:
validation = label_ec(validation)

Se muestra el classification report.

In [40]:
print(classification_report(label_validation, xboost.predict(validation)))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99    114044
           1       0.74      0.39      0.51      4064

    accuracy                           0.97    118108
   macro avg       0.86      0.69      0.75    118108
weighted avg       0.97      0.97      0.97    118108



Calculo el ***roc auc*** del validation con el modelo manual.

In [41]:
roc_auc_score(label_validation, xboost.predict_proba(validation)[:,1])

0.8955829899935707

### Hiperparametros

Realizo la busqueda de hiperparametros utilizando RandomizedSearchCV de *Sklearn*.

In [42]:
"""from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
clf = xgb.XGBClassifier(objective='binary:logistic',random_state=0)
distributions = dict(n_estimators = [60,80,90,100,120,130,140,150,160,170,180,190,200],
                     max_depth = [3,4,5,6,7,8,9,10,11,12,13,14,15],
                     subsample =[0.6,0.7,0.8,0.9,1],
                     learning_rate=[0,0.05,0.1,0.15,0.20,0.25,0.3],
                     colsample_bytree=[0.85,0.86,0.87,0.88,0.89,0.9,0.91,0.92,0.93,0.94,0.95,0.96,0.97,0,98,0.99,1],
                     gamma =[0.85,0.86,0.87,0.88,0.89,0.9,0.91,0.92,0.93,0.94,0.95,0.96,0.97,0,98,0.99,1])
                     
rs = RandomizedSearchCV(clf, distributions, scoring='roc_auc',n_jobs=1, verbose=100, cv=3,n_iter=4, random_state=0)
search = rs.fit(train, label_train)
search.best_params_"""

"from sklearn.model_selection import RandomizedSearchCV\nfrom scipy.stats import uniform\nclf = xgb.XGBClassifier(objective='binary:logistic',random_state=0)\ndistributions = dict(n_estimators = [60,80,90,100,120,130,140,150,160,170,180,190,200],\n                     max_depth = [3,4,5,6,7,8,9,10,11,12,13,14,15],\n                     subsample =[0.6,0.7,0.8,0.9,1],\n                     learning_rate=[0,0.05,0.1,0.15,0.20,0.25,0.3],\n                     colsample_bytree=[0.85,0.86,0.87,0.88,0.89,0.9,0.91,0.92,0.93,0.94,0.95,0.96,0.97,0,98,0.99,1],\n                     gamma =[0.85,0.86,0.87,0.88,0.89,0.9,0.91,0.92,0.93,0.94,0.95,0.96,0.97,0,98,0.99,1])\n                     \nrs = RandomizedSearchCV(clf, distributions, scoring='roc_auc',n_jobs=1, verbose=100, cv=3,n_iter=4, random_state=0)\nsearch = rs.fit(train, label_train)\nsearch.best_params_"

Guardo el modelo de hiperparametros.

In [43]:
"""import pickle
with open('model_backup_punto_3_mod_1', 'wb') as model_backup_file:
    pickle.dump(search, model_backup_file)"""

"import pickle\nwith open('model_backup_punto_3_mod_1', 'wb') as model_backup_file:\n    pickle.dump(search, model_backup_file)"

Cargo el modelo con hiperparametros.

In [44]:
import pickle
with open('model_backup_punto_3_mod_1', 'rb') as file:
     search = pickle.load(file)

Se muestran los mejores parametros seleccionados por el RandomizedSearch de *Sklearn*.

In [45]:
search.best_params_

{'subsample': 0.6,
 'n_estimators': 140,
 'max_depth': 4,
 'learning_rate': 0.3,
 'gamma': 0.94,
 'colsample_bytree': 0.94}

In [46]:
print(classification_report(label_validation, search.predict(validation)))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99    114044
           1       0.67      0.39      0.49      4064

    accuracy                           0.97    118108
   macro avg       0.83      0.69      0.74    118108
weighted avg       0.97      0.97      0.97    118108



In [47]:
roc_auc_score(label_validation, search.predict_proba(validation)[:,1])

0.8928214106028147

### Respuesta
Para el mejor modelo de ambos, ¿cuál es el score en la competencia? (guardar el
csv con predicciones para entregarlo después)


Dado que el mejor modelo (el que me dio el roc auc mas alto para el validation) es el xgboost manual, es decir, el que hice en un primer momento sin la busqueda de hiperparametros, fue el que me dio un roc auc de 0.8955, que resulto el mas alto, lo utilizare para cargarlo a Kaggle. 

###### Leo Test para calcular el predict proba para la competencia

In [49]:
idenTest = pd.read_csv('test_identity.csv')

In [50]:
transTest = pd.read_csv('test_transaction.csv')

Renombro las cols dado que tienen nombre distintos a los df de train y validation.

In [52]:
def renombrar(df):
    for columna in df.columns:
        if columna.startswith('id'):
            ss = columna.split('-')
            df.rename(columns={columna:ss[0]+'_'+ss[1]},inplace=True)
    return (df)
        

In [53]:
idenTest = renombrar(idenTest)

Realizo el merge de los dos df.

In [54]:
test = transTest.merge(idenTest,how='left')

In [55]:
ids = test[['TransactionID']]

In [56]:
del(test['TransactionID'])

Realizo los encodings.

In [57]:
for i in countV:
    test=countVect(test,i,dict_encoding)

In [58]:
for i in oneHot:
    test = one_hot_encode(test,i,dict_encoding)

In [59]:
for i in mean:
    test = mean_encoding_val_test(test,i,dict_encoding)

In [60]:
test = label_ec(test)

Calculo el predict_proba

In [61]:
test['isFraud'] =xboost.predict_proba(test)[:,1]

In [62]:
test = test[['isFraud']]

In [63]:
join = ids.join(test)

Se visualiza el df que sera subido al Kaggle.

In [66]:
join

,TransactionID,isFraud
0,3663549,0.008020
1,3663550,0.005345
2,3663551,0.005456
3,3663552,0.001402
4,3663553,0.005215
...,...,...
506686,4170235,0.021423
506687,4170236,0.007643
506688,4170237,0.005882
506689,4170238,0.012973


In [67]:
join.to_csv("entregaKaggle_mejor_mod_punto_3.csv", index = False)

El *score* obtenido en la *competencia de Kaggle* para mi ***mejor modelo*** fue:
<img src = 'score-mejor-mod-punto-3.png' alt = "title">